In [1]:
# -*- coding: utf-8 -*-
from __future__ import unicode_literals
# text in Western (Windows 1252)

import pickle
import numpy as np
# import StringIO
import math
from keras import optimizers, metrics
from keras.models import Model
from keras.layers import Dense, Dropout, Input
from keras.layers.merge import concatenate
from keras import regularizers
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from keras.constraints import maxnorm
from keras.layers import Flatten
from keras.optimizers import SGD
from keras.models import load_model
# from keras import backend as Input
np.random.seed(7)

Using Theano backend.


In [2]:
# %run ../../../prepare_data.py

import sys
sys.path.insert(0, '../../../')
from prepare_data import *

In [3]:
# %run ../../../prepare_data.py
# X_train, X_other_features_train, y_train, X_test, X_other_features_test, y_test, X_validate, X_other_features_validate, y_validate = generate_full_matrix_inputs('../../internal_representations/inputs/content_shuffle_vector.h5', '../../internal_representations/inputs/shuffle_vector')
# save_inputs('../../internal_representations/inputs/shuffled_letters_train.h5', X_train, y_train, other_features = X_other_features_train)
# save_inputs('../../internal_representations/inputs/shuffled_letters_test.h5', X_test, y_test, other_features = X_other_features_test)
# save_inputs('../../internal_representations/inputs/shuffled_letters_validate.h5', X_validate, y_validate,  other_features = X_other_features_validate)
# X_train, X_other_features_train, y_train = load_inputs('../../internal_representations/inputs/shuffled_letters_train.h5', other_features=True)
# X_test, X_other_features_test, y_test = load_inputs('../../internal_representations/inputs/shuffled_letters_test.h5', other_features=True)
# X_validate, X_other_features_validate, y_validate = load_inputs('../../internal_representations/inputs/shuffled_letters_validate.h5', other_features=True)

data = Data('l', accent_classification=True)
data.generate_data('letters_accent_classification_train',
                   'letters_accent_classification_test',
                   'letters_accent_classification_validate', force_override=False)

LOADING DATA...
LOAD SUCCESSFUL!


In [123]:
gen2 = data.generator('train', 16)
# test1 = next(gen1)
# test2 = next(gen2)

In [102]:
test2 = next(gen2)

In [129]:
test2 = next(gen2)
pos = 0
print(len(feature_dictionary))
print(np.array(test2[0][0]).shape)
print(np.array(test2[0][1]).shape)
print(len(test2[0]))
print(len(test2))
for el in test2[0][0]:
#     print(el)
    print (data.decode_x(el, dictionary))
#     print(data.decode_x_other_features(feature_dictionary, [el]))

# for el in test2[1]:
#     print(el)

BBB
10
(16, 23, 36)
(16, 150)
2
2
invitagen
iremirp
iksneko
jnavolimop
mišjentsačilevjan
mišjentsačilevjan
hišjenbordopjan
hišjenbordopjan
enitesjavd
amonobarab
enejlugo
amtsonlanoiseforp
anavokilboen
anavokilboen
ebžalot
invejark


In [6]:
def count_vowels(content, accented_vowels):
    num_all_vowels = 0
    for el in content:
        for m in list(el[3]):
            if m in accented_vowels:
                num_all_vowels += 1
    return num_all_vowels

count_vowels(content, accented_vowels)

592886

In [10]:
# print (X_train.shape)
# print (X_test.shape)
# print (X_validate.shape)
pos = 0
# print (decode_input(X_train[pos], dictionary))
# print (decode_X_features(feature_dictionary, [X_other_features_train[pos]]))
# print(decode_position(y_train[pos]))
# print('------------------------------------')
print (data.x_train.shape)
print (data.x_test.shape)
print (data.x_validate.shape)
# pos = 2
print (data.decode_x(data.x_train[pos], dictionary))
print (data.decode_x_other_features(feature_dictionary, [data.x_other_features_train[pos]]))
print(data.decode_y(data.y_train[pos]))
data.y_train[pos]

(430151, 23, 36)
(52058, 23, 36)
(54222, 23, 36)
ašjenlautkajan
Agsnpn-
Agsnpn-
[2, 5]


array([ 0.,  0.,  1.,  0.,  0.,  2.,  0.,  0.,  0.,  0.])

In [4]:
# num_examples = count_vowels(content, accented_vowels) # training set size
num_examples = 592886
nn_output_dim = 13
nn_hdim = 516
batch_size = 16
actual_epoch = 60
num_fake_epoch = 20


# num_examples = len(data.x_train) # training set size
# nn_output_dim = 11
# nn_hdim = 516
# batch_size = 16
# actual_epoch = 1
# num_fake_epoch = 20



conv_input_shape=(23, 36)
othr_input = (150, )

conv_input = Input(shape=conv_input_shape, name='conv_input')
x_conv = Conv1D(69, (3), padding='same', activation='relu')(conv_input)
x_conv = Conv1D(43, (3), padding='same', activation='relu')(x_conv)
x_conv = MaxPooling1D(pool_size=2)(x_conv)
x_conv = Flatten()(x_conv)
# x_conv = Dense(516, activation='relu', kernel_constraint=maxnorm(3))(x_conv)

othr_input = Input(shape=othr_input, name='othr_input')
# x_othr = Dense(256, input_dim=167, activation='relu')(othr_input)
# x_othr = Dropout(0.3)(x_othr)
# x_othr = Dense(512, activation='relu')(othr_input)
# x_othr = Dropout(0.3)(x_othr)
# x_othr = Dense(256, activation='relu')(othr_input)

x = concatenate([x_conv, othr_input])
# x = Dense(1024, input_dim=(516 + 256), activation='relu')(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.3)(x)
x = Dense(512, activation='relu')(x)
x = Dropout(0.3)(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.2)(x)
x = Dense(nn_output_dim, activation='sigmoid')(x)




model = Model(inputs=[conv_input, othr_input], outputs=x)
opt = optimizers.Adam(lr=1E-4, beta_1=0.9, beta_2=0.999, epsilon=1e-08)
model.compile(loss='binary_crossentropy', optimizer=opt, metrics=[actual_accuracy,])
# model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])

INFO (theano.gof.compilelock): Waiting for existing lock by unknown process (I am process '1158')
INFO (theano.gof.compilelock): To manually release the lock, delete /home/lukakrsnik/.theano/compiledir_Linux-4.4--generic-x86_64-with-debian-stretch-sid-x86_64-3.5.2-64/lock_dir


In [5]:
history = model.fit_generator(data.generator('train', batch_size), 
                              num_examples/(batch_size * num_fake_epoch), 
                              epochs=actual_epoch*num_fake_epoch, 
                              validation_data=data.generator('test', batch_size), 
                              validation_steps=num_examples/(batch_size * num_fake_epoch))

Epoch 1/1200
1853/1852 [==============================] - 58s - loss: 0.2146 - actual_accuracy: 0.0882 - val_loss: 0.1655 - val_actual_accuracy: 0.2487
Epoch 2/1200
1853/1852 [==============================] - 49s - loss: 0.1476 - actual_accuracy: 0.3643 - val_loss: 0.1162 - val_actual_accuracy: 0.5555
Epoch 3/1200
1853/1852 [==============================] - 63s - loss: 0.1066 - actual_accuracy: 0.5835 - val_loss: 0.0803 - val_actual_accuracy: 0.7170
Epoch 4/1200
1853/1852 [==============================] - 56s - loss: 0.0769 - actual_accuracy: 0.7316 - val_loss: 0.0569 - val_actual_accuracy: 0.8231
Epoch 5/1200
1853/1852 [==============================] - 55s - loss: 0.0610 - actual_accuracy: 0.8010 - val_loss: 0.0448 - val_actual_accuracy: 0.8683
Epoch 6/1200
1853/1852 [==============================] - 64s - loss: 0.0508 - actual_accuracy: 0.8404 - val_loss: 0.0396 - val_actual_accuracy: 0.8841
Epoch 7/1200
1853/1852 [==============================] - 62s - loss: 0.0464 - actual_ac

1853/1852 [==============================] - 58s - loss: 0.0049 - actual_accuracy: 0.9857 - val_loss: 0.0287 - val_actual_accuracy: 0.9628
Epoch 108/1200
1853/1852 [==============================] - 59s - loss: 0.0048 - actual_accuracy: 0.9868 - val_loss: 0.0302 - val_actual_accuracy: 0.9581
Epoch 109/1200
1853/1852 [==============================] - 58s - loss: 0.0049 - actual_accuracy: 0.9862 - val_loss: 0.0288 - val_actual_accuracy: 0.9599
Epoch 110/1200
1853/1852 [==============================] - 56s - loss: 0.0048 - actual_accuracy: 0.9864 - val_loss: 0.0262 - val_actual_accuracy: 0.9645
Epoch 111/1200
1853/1852 [==============================] - 57s - loss: 0.0048 - actual_accuracy: 0.9860 - val_loss: 0.0291 - val_actual_accuracy: 0.9590
Epoch 112/1200
1853/1852 [==============================] - 62s - loss: 0.0051 - actual_accuracy: 0.9855 - val_loss: 0.0246 - val_actual_accuracy: 0.9611
Epoch 113/1200
1853/1852 [==============================] - 60s - loss: 0.0048 - actual_acc

1853/1852 [==============================] - 61s - loss: 0.0027 - actual_accuracy: 0.9924 - val_loss: 0.0306 - val_actual_accuracy: 0.9629
Epoch 214/1200
1853/1852 [==============================] - 64s - loss: 0.0030 - actual_accuracy: 0.9910 - val_loss: 0.0307 - val_actual_accuracy: 0.9623
Epoch 215/1200
1853/1852 [==============================] - 59s - loss: 0.0031 - actual_accuracy: 0.9905 - val_loss: 0.0296 - val_actual_accuracy: 0.9647
Epoch 216/1200
1853/1852 [==============================] - 61s - loss: 0.0029 - actual_accuracy: 0.9917 - val_loss: 0.0326 - val_actual_accuracy: 0.9610
Epoch 217/1200
1853/1852 [==============================] - 65s - loss: 0.0029 - actual_accuracy: 0.9919 - val_loss: 0.0280 - val_actual_accuracy: 0.9655
Epoch 218/1200
1853/1852 [==============================] - 60s - loss: 0.0030 - actual_accuracy: 0.9918 - val_loss: 0.0314 - val_actual_accuracy: 0.9627
Epoch 219/1200
1853/1852 [==============================] - 57s - loss: 0.0031 - actual_acc

1853/1852 [==============================] - 63s - loss: 0.0025 - actual_accuracy: 0.9933 - val_loss: 0.0328 - val_actual_accuracy: 0.9610
Epoch 320/1200
1853/1852 [==============================] - 65s - loss: 0.0028 - actual_accuracy: 0.9930 - val_loss: 0.0336 - val_actual_accuracy: 0.9623
Epoch 321/1200
1853/1852 [==============================] - 61s - loss: 0.0024 - actual_accuracy: 0.9935 - val_loss: 0.0333 - val_actual_accuracy: 0.96420.99
Epoch 322/1200
1853/1852 [==============================] - 63s - loss: 0.0024 - actual_accuracy: 0.9938 - val_loss: 0.0315 - val_actual_accuracy: 0.9618
Epoch 323/1200
1853/1852 [==============================] - 59s - loss: 0.0022 - actual_accuracy: 0.9939 - val_loss: 0.0344 - val_actual_accuracy: 0.9644
Epoch 324/1200
1853/1852 [==============================] - 63s - loss: 0.0022 - actual_accuracy: 0.9936 - val_loss: 0.0321 - val_actual_accuracy: 0.9653 0.002
Epoch 325/1200
1853/1852 [==============================] - 58s - loss: 0.0026 - 

1853/1852 [==============================] - 64s - loss: 0.0021 - actual_accuracy: 0.9947 - val_loss: 0.0344 - val_actual_accuracy: 0.9621
Epoch 426/1200
1853/1852 [==============================] - 62s - loss: 0.0022 - actual_accuracy: 0.9947 - val_loss: 0.0318 - val_actual_accuracy: 0.9659
Epoch 427/1200
1853/1852 [==============================] - 62s - loss: 0.0023 - actual_accuracy: 0.9939 - val_loss: 0.0340 - val_actual_accuracy: 0.9627
Epoch 428/1200
1853/1852 [==============================] - 65s - loss: 0.0023 - actual_accuracy: 0.9936 - val_loss: 0.0344 - val_actual_accuracy: 0.9627
Epoch 429/1200
1853/1852 [==============================] - 60s - loss: 0.0020 - actual_accuracy: 0.9944 - val_loss: 0.0307 - val_actual_accuracy: 0.9655
Epoch 430/1200
1853/1852 [==============================] - 65s - loss: 0.0022 - actual_accuracy: 0.9943 - val_loss: 0.0331 - val_actual_accuracy: 0.9643
Epoch 431/1200
1853/1852 [==============================] - 61s - loss: 0.0025 - actual_acc

...

1853/1852 [==============================] - 80s - loss: 0.0022 - actual_accuracy: 0.9947 - val_loss: 0.0312 - val_actual_accuracy: 0.9648
Epoch 743/1200
1853/1852 [==============================] - 80s - loss: 0.0020 - actual_accuracy: 0.9950 - val_loss: 0.0302 - val_actual_accuracy: 0.9679
Epoch 744/1200
1853/1852 [==============================] - 85s - loss: 0.0023 - actual_accuracy: 0.9944 - val_loss: 0.0305 - val_actual_accuracy: 0.9673
Epoch 745/1200
1853/1852 [==============================] - 84s - loss: 0.0021 - actual_accuracy: 0.9948 - val_loss: 0.0321 - val_actual_accuracy: 0.9646
Epoch 746/1200
1853/1852 [==============================] - 78s - loss: 0.0017 - actual_accuracy: 0.9952 - val_loss: 0.0299 - val_actual_accuracy: 0.9690
Epoch 747/1200
1853/1852 [==============================] - 85s - loss: 0.0019 - actual_accuracy: 0.9951 - val_loss: 0.0344 - val_actual_accuracy: 0.9633
Epoch 748/1200
1853/1852 [==============================] - 83s - loss: 0.0022 - actual_acc

1853/1852 [==============================] - 76s - loss: 0.0021 - actual_accuracy: 0.9947 - val_loss: 0.0331 - val_actual_accuracy: 0.9630
Epoch 848/1200
1853/1852 [==============================] - 74s - loss: 0.0019 - actual_accuracy: 0.9954 - val_loss: 0.0316 - val_actual_accuracy: 0.9679
Epoch 849/1200
1853/1852 [==============================] - 69s - loss: 0.0019 - actual_accuracy: 0.9956 - val_loss: 0.0372 - val_actual_accuracy: 0.9615
Epoch 850/1200
1853/1852 [==============================] - 70s - loss: 0.0019 - actual_accuracy: 0.9950 - val_loss: 0.0327 - val_actual_accuracy: 0.9667
Epoch 851/1200
1853/1852 [==============================] - 73s - loss: 0.0023 - actual_accuracy: 0.9956 - val_loss: 0.0336 - val_actual_accuracy: 0.9668
Epoch 852/1200
1853/1852 [==============================] - 74s - loss: 0.0017 - actual_accuracy: 0.9957 - val_loss: 0.0332 - val_actual_accuracy: 0.9665
Epoch 853/1200
1853/1852 [==============================] - 77s - loss: 0.0020 - actual_acc

1853/1852 [==============================] - 68s - loss: 0.0023 - actual_accuracy: 0.9937 - val_loss: 0.0342 - val_actual_accuracy: 0.9617
Epoch 953/1200
1853/1852 [==============================] - 68s - loss: 0.0021 - actual_accuracy: 0.9947 - val_loss: 0.0298 - val_actual_accuracy: 0.9686
Epoch 954/1200
1853/1852 [==============================] - 74s - loss: 0.0017 - actual_accuracy: 0.9963 - val_loss: 0.0349 - val_actual_accuracy: 0.9663
Epoch 955/1200
1853/1852 [==============================] - 84s - loss: 0.0021 - actual_accuracy: 0.9947 - val_loss: 0.0348 - val_actual_accuracy: 0.9648.9
Epoch 956/1200
1853/1852 [==============================] - 81s - loss: 0.0016 - actual_accuracy: 0.9959 - val_loss: 0.0376 - val_actual_accuracy: 0.9653
Epoch 957/1200
1853/1852 [==============================] - 73s - loss: 0.0020 - actual_accuracy: 0.9949 - val_loss: 0.0331 - val_actual_accuracy: 0.9662
Epoch 958/1200
1853/1852 [==============================] - 84s - loss: 0.0018 - actual_a

1853/1852 [==============================] - 76s - loss: 0.0019 - actual_accuracy: 0.9951 - val_loss: 0.0335 - val_actual_accuracy: 0.9659
Epoch 1057/1200
1853/1852 [==============================] - 74s - loss: 0.0022 - actual_accuracy: 0.9949 - val_loss: 0.0350 - val_actual_accuracy: 0.9658
Epoch 1058/1200
1853/1852 [==============================] - 64s - loss: 0.0021 - actual_accuracy: 0.9948 - val_loss: 0.0342 - val_actual_accuracy: 0.9642
Epoch 1059/1200
1853/1852 [==============================] - 76s - loss: 0.0024 - actual_accuracy: 0.9946 - val_loss: 0.0369 - val_actual_accuracy: 0.9650
Epoch 1060/1200
1853/1852 [==============================] - 83s - loss: 0.0023 - actual_accuracy: 0.9943 - val_loss: 0.0344 - val_actual_accuracy: 0.9655
Epoch 1061/1200
1853/1852 [==============================] - 81s - loss: 0.0018 - actual_accuracy: 0.9952 - val_loss: 0.0441 - val_actual_accuracy: 0.9594
Epoch 1062/1200
1853/1852 [==============================] - 73s - loss: 0.0019 - actu

1853/1852 [==============================] - 72s - loss: 0.0018 - actual_accuracy: 0.9954 - val_loss: 0.0292 - val_actual_accuracy: 0.9680
Epoch 1161/1200
1853/1852 [==============================] - 70s - loss: 0.0016 - actual_accuracy: 0.9959 - val_loss: 0.0305 - val_actual_accuracy: 0.9669
Epoch 1162/1200
1853/1852 [==============================] - 69s - loss: 0.0019 - actual_accuracy: 0.9955 - val_loss: 0.0355 - val_actual_accuracy: 0.9670
Epoch 1163/1200
1853/1852 [==============================] - 74s - loss: 0.0025 - actual_accuracy: 0.9942 - val_loss: 0.0308 - val_actual_accuracy: 0.9597
Epoch 1164/1200
1853/1852 [==============================] - 72s - loss: 0.0026 - actual_accuracy: 0.9941 - val_loss: 0.0402 - val_actual_accuracy: 0.9585
Epoch 1165/1200
1853/1852 [==============================] - 72s - loss: 0.0022 - actual_accuracy: 0.9955 - val_loss: 0.0324 - val_actual_accuracy: 0.9653
Epoch 1166/1200
1853/1852 [==============================] - 66s - loss: 0.0024 - actu

In [6]:
name = '60_epoch'
model.save(name + '.h5')
output = open(name + '_history.pkl', 'wb')
pickle.dump(history.history, output)
output.close()

In [9]:
content = data._read_content('../../../data/SlovarIJS_BESEDE_utf8.lex')
dictionary, max_word, max_num_vowels, vowels, accented_vowels = data._create_dict(content)
feature_dictionary = data._create_feature_dictionary()